In [18]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import accuracy_score, classification_report
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
import numpy as np

In [19]:
df = pd.read_csv("pre_processed_data/mfcc52_oversampled.csv") 

In [20]:

df.reset_index(drop=True, inplace = True)

In [21]:
df_x = df.iloc[:,:52]
df_y = df.iloc[:,-1]

In [22]:
x = np.array(df_x).reshape(-1, 52, 1)

In [23]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(df_y)

In [24]:
unique_elements, counts_elements = np.unique(y_encoded, return_counts=True)
print(np.asarray((unique_elements, counts_elements)))

[[  0   1   2   3   4]
 [222 222 222 222 222]]


In [25]:
x_train, x_test, y_train, y_test = train_test_split(x, y_encoded, 
                                                    test_size=0.2, random_state = 42)

In [26]:
# Create the model
model = Sequential()

# Add convolutional layers
model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(52, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

# Flatten the output of the convolutional layers
model.add(Flatten())

# Add dense layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(5, activation='softmax')) 


# Compile the model
optimiser = tf.keras.optimizers.Adam(learning_rate = 0.0002)
cb = [EarlyStopping(patience=10,monitor='accuracy',mode='max',restore_best_weights=True)]
model.compile(optimizer=optimiser, loss='sparse_categorical_crossentropy', metrics = ['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=100, batch_size=16, validation_data=(x_test, y_test), callbacks = cb)

c:\Users\Yash Thakar\PROGRAMMING\Quantum\env\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.1970 - loss: 2.6887 - val_accuracy: 0.3784 - val_loss: 1.5537
Epoch 2/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.2286 - loss: 1.6523 - val_accuracy: 0.4189 - val_loss: 1.5091
Epoch 3/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.2656 - loss: 1.5703 - val_accuracy: 0.3694 - val_loss: 1.4618
Epoch 4/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.3280 - loss: 1.4967 - val_accuracy: 0.4009 - val_loss: 1.3922
Epoch 5/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.3283 - loss: 1.4773 - val_accuracy: 0.4279 - val_loss: 1.3716
Epoch 6/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.3406 - loss: 1.4214 - val_accuracy: 0.5090 - val_loss: 1.3274
Epoch 7/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.3679 - loss: 1.3662 - val_accuracy: 0.5180 - val_loss: 1.2710
Epoch 8/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.4175 - loss: 1.3405 - val_accuracy: 0.50

In [27]:
y_pred = model.predict(x_test)
y_pred_classes = y_pred.argmax(axis=1)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


In [28]:
y_pred

array([[2.56743398e-04, 1.46720195e-05, 9.97036338e-01, 3.09284078e-04,
        2.38289079e-03],
       [9.99982238e-01, 1.84492819e-06, 1.27340945e-05, 1.78540336e-06,
        1.37390327e-06],
       [6.28137171e-01, 1.41871171e-02, 4.56015160e-03, 3.68045294e-03,
        3.49435061e-01],
       ...,
       [6.32645607e-01, 1.36363255e-02, 4.50696936e-03, 3.52036324e-03,
        3.45690817e-01],
       [1.84022918e-01, 4.09883857e-02, 2.31951728e-01, 2.68571794e-01,
        2.74465144e-01],
       [3.56194400e-03, 5.33449173e-04, 6.75492594e-03, 2.03602976e-04,
        9.88946021e-01]], dtype=float32)

In [29]:
y_pred_classes

array([2, 0, 0, 0, 0, 3, 0, 3, 0, 3, 2, 3, 1, 2, 0, 2, 0, 1, 1, 0, 2, 0,
       1, 1, 0, 3, 4, 0, 2, 0, 1, 2, 3, 3, 3, 1, 2, 0, 0, 1, 1, 4, 3, 2,
       1, 1, 3, 3, 1, 4, 3, 3, 4, 3, 2, 1, 3, 1, 2, 0, 0, 0, 1, 1, 4, 1,
       2, 0, 4, 3, 0, 0, 4, 0, 1, 3, 3, 0, 3, 1, 2, 1, 0, 2, 1, 2, 4, 4,
       1, 3, 1, 4, 3, 2, 3, 3, 4, 4, 1, 2, 1, 3, 0, 4, 4, 0, 0, 3, 4, 4,
       4, 0, 4, 2, 3, 0, 1, 0, 3, 3, 3, 0, 3, 4, 4, 1, 4, 0, 0, 0, 0, 2,
       2, 2, 4, 0, 0, 4, 1, 1, 1, 4, 3, 3, 3, 2, 1, 3, 3, 2, 0, 1, 1, 0,
       3, 3, 2, 3, 2, 3, 4, 2, 0, 0, 0, 1, 0, 2, 2, 1, 3, 3, 1, 2, 4, 2,
       2, 1, 4, 4, 0, 4, 3, 2, 4, 0, 0, 3, 2, 3, 1, 2, 1, 1, 3, 0, 3, 0,
       2, 4, 2, 2, 2, 3, 3, 2, 3, 2, 0, 0, 0, 2, 2, 3, 0, 4, 1, 3, 0, 0,
       4, 4], dtype=int64)

In [30]:
accuracy = accuracy_score(y_test, y_pred_classes)
print(f'Accuracy Score: {accuracy}')

Accuracy Score: 0.8468468468468469


In [31]:
class_report = classification_report(y_test, y_pred_classes)
print('Classification Report:\n', class_report)

Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.87      0.86        53
           1       1.00      0.93      0.96        44
           2       0.81      0.76      0.78        45
           3       0.78      0.87      0.82        46
           4       0.79      0.79      0.79        34

    accuracy                           0.85       222
   macro avg       0.85      0.84      0.84       222
weighted avg       0.85      0.85      0.85       222



In [32]:
y_pred = model.predict(x_train)
y_pred_classes = y_pred.argmax(axis=1)

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


In [33]:
accuracy = accuracy_score(y_train, y_pred_classes)
print(f'Accuracy Score: {accuracy}')

Accuracy Score: 0.9009009009009009


In [34]:
class_report = classification_report(y_train, y_pred_classes)
print('Classification Report:\n', class_report)

Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.95      0.90       169
           1       1.00      0.97      0.99       178
           2       0.91      0.84      0.87       177
           3       0.84      0.92      0.88       176
           4       0.91      0.83      0.87       188

    accuracy                           0.90       888
   macro avg       0.90      0.90      0.90       888
weighted avg       0.90      0.90      0.90       888

